In [15]:
import math

# Функция активации и её производная (ReLU и сигмоид)
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

def relu(x):
    return max(0, x)

def relu_derivative(x):
    return 1 if x > 0 else 0

# Инициализация весов с фиксированными значениями
def init_fixed_weights(rows, cols, value=0.25):
    return [[value+0.1*i for i in range(cols)] for _ in range(rows)]

# Класс нейронной сети
class SimpleNeuralNetwork:
    def __init__(self):
        # Задание фиксированных размеров сети
        self.input_size = 2
        self.hidden_size = 5
        self.output_size = 1

        # Веса между входным слоем и первым скрытым слоем
        self.weights1 = init_fixed_weights(self.input_size, self.hidden_size, value=0.5)
        self.bias1 = [-0.5+0.2*i for i in range(self.hidden_size)]

        # Веса между первым и вторым скрытым слоем
        self.weights2 = init_fixed_weights(self.hidden_size, self.hidden_size, value=0.1)
        self.bias2 = [-0.25+0.1*i for i in range(self.hidden_size)]

        # Веса между вторым скрытым слоем и выходом
        self.weights3 = init_fixed_weights(self.hidden_size, self.output_size, value=-0.5)
        self.bias3 = [-0.05+0.05*i for i in range(self.output_size)]

    # Прямой проход (feedforward)
    def feedforward(self, x):
        # Входной слой -> Первый скрытый слой
        self.hidden1 = [relu(sum(x[j] * self.weights1[j][i] for j in range(self.input_size)) + self.bias1[i]) for i in range(self.hidden_size)]

        # Первый скрытый слой -> Второй скрытый слой
        self.hidden2 = [relu(sum(self.hidden1[j] * self.weights2[j][i] for j in range(self.hidden_size)) + self.bias2[i]) for i in range(self.hidden_size)]

        # Второй скрытый слой -> Выходной слой
        self.output = [sigmoid(sum(self.hidden2[j] * self.weights3[j][i] for j in range(self.hidden_size)) + self.bias3[i]) for i in range(self.output_size)]
        
        return self.output[0]

    # Обратное распространение ошибки
    def backpropagate(self, x, y, learning_rate):
        # Ошибка на выходе
        output_error = (self.output[0] - y) * sigmoid_derivative(self.output[0])

        # Градиенты для weights3 и bias3
        d_weights3 = [[output_error * self.hidden2[j] for j in range(self.hidden_size)]]
        d_bias3 = [output_error]

        # Ошибка для второго скрытого слоя
        hidden2_error = [output_error * self.weights3[j][0] * relu_derivative(self.hidden2[j]) for j in range(self.hidden_size)]

        # Градиенты для weights2 и bias2
        d_weights2 = [[hidden2_error[i] * self.hidden1[j] for j in range(self.hidden_size)] for i in range(self.hidden_size)]
        d_bias2 = hidden2_error

        # Ошибка для первого скрытого слоя
        hidden1_error = [sum(hidden2_error[k] * self.weights2[i][k] for k in range(self.hidden_size)) * relu_derivative(self.hidden1[i]) for i in range(self.hidden_size)]

        # Градиенты для weights1 и bias1
        d_weights1 = [[hidden1_error[i] * x[j] for j in range(self.input_size)] for i in range(self.hidden_size)]
        d_bias1 = hidden1_error

        # Обновление весов и смещений
        for i in range(self.hidden_size):
            for j in range(self.output_size):
                self.weights3[i][j] -= learning_rate * d_weights3[0][i]
            self.bias3[j] -= learning_rate * d_bias3[j]

        for i in range(self.hidden_size):
            for j in range(self.hidden_size):
                self.weights2[i][j] -= learning_rate * d_weights2[i][j]
            self.bias2[i] -= learning_rate * d_bias2[i]

        for i in range(self.hidden_size):
            for j in range(self.input_size):
                self.weights1[j][i] -= learning_rate * d_weights1[i][j]
            self.bias1[i] -= learning_rate * d_bias1[i]

    # Функция обучения
    def train(self, data, labels, epochs, learning_rate):
        for epoch in range(epochs):
            for x, y in zip(data, labels):
                self.feedforward(x)
                self.backpropagate(x, y, learning_rate)

# Пример данных для обучения
data = [[0, 0], [0, 1], [1, 0], [1, 1]]
labels = [0, 1, 1, 0]  

# Обучение
nn = SimpleNeuralNetwork()
nn.train(data, labels, epochs=10000, learning_rate=0.01)

# Тест
for x in data:
    print(f"Input: {x}, Predicted Output: {nn.feedforward(x):.4f}")

Input: [0, 0], Predicted Output: 0.0617
Input: [0, 1], Predicted Output: 0.9638
Input: [1, 0], Predicted Output: 0.9637
Input: [1, 1], Predicted Output: 0.0285
